# Assignment 2

Deadline: 26.03.2025, 12:00 CET

*Buchkov Viacheslav, ID = 24-742-488, viacheslav.buchkov@uzh.ch*

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [1]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

# Import local modules
project_root = os.path.dirname(
    os.path.dirname(os.getcwd())
)  # Change this path if needed
src_path = os.path.join(project_root, "qpmwp-course\\src")
sys.path.append(project_root)
sys.path.append(src_path)
from src.estimation.covariance import Covariance
from src.estimation.expected_return import ExpectedReturn
from src.optimization.constraints import Constraints
from src.optimization.quadratic_program import QuadraticProgram
from src.helper_functions import load_data_msci

### Derivation of 2-fold constraint:

We have $w = w^0 + w^+ - w^-$. Denote the auxiliary variable as $w^{aux} = w^+ + w^-$.

1. Rewrite the constraint from $\sum |w - w^0| <= \tau$ into:
    * $\sum w^{aux} <= \tau$
    * $w^{aux} >= w - w^0$
    * $w^aux >= -(w - w^0) = w^0 - w$.
2. Using the definition of auxiliary variable rewrite weights into the system:
$$
\begin{equation}
    \begin{cases}
      w = w^0 + w^{aux} - 2 w^- \\
      w = w^0 + 2 w^+ - w^{aux}
    \end{cases}\,.
\end{equation}
$$
3. Next use the restriction on the $w^+$ and $w^-$, denoting $upper\_bound = ub$ and $lower\_bound = lb$:
$$
\begin{equation}
    \begin{cases}
      0 \leq w^+ \leq ub - w^0 \\
      0 \leq w^- \leq w^0 - lb
    \end{cases}\,.
\end{equation}
$$.
4. Applying the above restrictions we obtain the following system on $w$ and $w^{aux}$:
$$
\begin{equation}
    \begin{cases}
      -w^0 + 2lb \leq w - w^{aux} \leq w^0 \\
      w^0 \leq w + w^{aux} \leq 2ub - w^0
    \end{cases}\,.
\end{equation}
$$.
5. Changing the inequality orders and rewriting we get 4 final inequalities:
$$
\begin{equation}
    \begin{cases}
      -w + w^{aux} \leq w^0 - 2lb \\
      w - w^{aux} \leq w^0 \\
      -w - w^{aux} \leq -w^0 \\
      w + w^{aux} \leq 2ub - w^0
    \end{cases}\,.
\end{equation}
$$.
6. Finally, as our $\widetilde{w} = (w, w^{aux})$ we augment the $\mathbb{G}$ and $\mathbb{h}$ as (with 5. meaning inequalities from 5.):
$$
\begin{equation}
    \begin{cases}
      \widetilde{\mathbb{G}} = (G, 5., \mathbb{1}_n) \\
      \widetilde{\mathbb{h}} = (h, 5., \tau)
    \end{cases}\,.
\end{equation}
$$.

In [2]:
def linearize_turnover_constraint(
    self, x_init: np.ndarray, to_budget: float = float("inf")
) -> None:
    """
    Linearize the turnover constraint in the quadratic programming problem with 2-fold augmentation.

    This method modifies the quadratic programming problem to include a linearized turnover constraint.

    For 2-fold implementation the auxiliary variable is taken as w^aux = w^+ + w^-. The linearization is
    done with G augmentation to transform sum |w - w_init| <= `to budget` into:
    * sum w^aux <= `to budget`
    * w^aux >= w - w_init
    * w^aux >= -(w - w_init).

    Parameters:
    -----------
    x_init : np.ndarray
        The initial portfolio weights.
    to_budget : float, optional
        The maximum allowable turnover. Defaults to float('inf').

    Notes:
    ------
    - The method updates the problem's objective function coefficients, inequality constraints,
    equality constraints, and bounds to account for the turnover constraint.
    - The original problem data is overridden with the updated matrices and vectors.

    Examples:
    ---------
    >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
    >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
    """
    # Dimensions
    n = len(self.problem_data.get("q"))
    m = 0 if self.problem_data.get("G") is None else self.problem_data.get("G").shape[0]

    # 1. Update the coefficients of the objective function
    P_orig = self.problem_data.get("P")
    q_orig = self.problem_data.get("q")

    # Augment by sum of positive and negative parts => only n auxiliary variables
    # that do not contribute to the mean-variance optim.
    P = np.pad(P_orig, ((0, n), (0, n)), constant_values=0)
    q = np.pad(q_orig, (0, n), constant_values=0)

    # 2. Update the equality constraints
    # As all the restrictions are done in step 3., just pad with n zeros for w^aux
    A_orig = self.problem_data.get("A")
    A = np.pad(A_orig, ((0, 0), (0, n)), constant_values=0)

    # 3. Update the inequality constraints
    G_orig = self.problem_data.get("G")
    h_orig = self.problem_data.get("h")

    lb_orig = self.problem_data.get("lb")
    ub_orig = self.problem_data.get("ub")

    # Have of the form Gw <= h
    # See the derivation above for the G and h augmentation logic
    G = np.zeros((4 * n + 1, 2 * n))

    def _fill_G(
        from_idx: int, to_idx: int, array_left: np.array, array_right: np.array
    ) -> None:
        G[from_idx:to_idx, :n] = array_left
        G[from_idx:to_idx, n:] = array_right

    # -w + w^aux
    _fill_G(0, n, np.eye(n) * -1, np.eye(n))
    # w - w^aux
    _fill_G(n, 2 * n, np.eye(n), np.eye(n) * -1)
    # -w - w^aux
    _fill_G(2 * n, 3 * n, np.eye(n) * -1, np.eye(n) * -1)
    # w + w^aux
    _fill_G(0, n, np.eye(n) * -1, np.eye(n))

    # \sum w^aux <= `to_budget`
    G[-1, n:] = np.ones(n)

    h = np.concatenate((x_init - 2 * lb_orig, x_init, -x_init, 2 * ub_orig - x_init))
    h = np.concatenate((h, np.array([to_budget])))

    if G_orig is not None:
        # Pad otherwise
        G_pad_auxiliary = np.pad(G_orig, ((0, 0), (0, n)), constant_values=0)
        G = np.concatenate((G_pad_auxiliary, G))

        h = np.concatenate((h_orig, h))

    # Update lower and upper bounds
    # As we take w = w^0 + w^+ - w^- => w^- is positive => all are positive
    lb = np.pad(lb_orig, (0, n), constant_values=0)

    ub = np.pad(ub_orig, (0, n), constant_values=0)
    ub[n:] = ub_orig - lb_orig

    # Override the original matrices
    self.update_problem_data(
        {"P": P, "q": q, "G": G, "h": h, "A": A, "lb": lb, "ub": ub}
    )

    return None

## Demo

#### Create P and q

In [3]:
# Load the msci country index data
N = 10
data = load_data_msci(path="../data/", n=N)
X = data["return_series"]

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method="geometric").estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method="pearson").estimate(X=X, inplace=False)

# q, P

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [4]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids=X.columns.tolist())
constraints.add_budget(rhs=1, sense="=")
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P=P.to_numpy(),
    q=q.to_numpy() * 0,
    G=GhAb["G"],
    h=GhAb["h"],
    A=GhAb["A"],
    b=GhAb["b"],
    lb=constraints.box["lower"].to_numpy(),
    ub=constraints.box["upper"].to_numpy(),
    solver="cvxopt",
)

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)

### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [5]:
MAX_TURNOVER = 0.5

# Prepare initial weights
x_init = pd.Series([1 / X.shape[1]] * X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=MAX_TURNOVER)

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get("solution")
ids = constraints.ids
weights = pd.Series(solution.x[: len(ids)], index=ids, name="optimal_weights")

turnover = np.abs(weights - x_init).sum()
volatility = np.sqrt(weights.to_numpy().T @ P.to_numpy() @ weights.to_numpy())

assert turnover <= MAX_TURNOVER, (
    f"Turnover constraint not respected: {turnover} > {MAX_TURNOVER}"
)

print("Turnover:")
print(turnover)

print("Volatility:")
print(volatility)

Turnover:
0.49989303208538993
Volatility:
0.008832098798467157


In [6]:
pd.concat([x_init.rename("initial_weights"), weights], axis=1)

,initial_weights,optimal_weights
AT,0.1,0.083752
AU,0.1,0.341020
BE,0.1,0.099981
CA,0.1,0.108878
CH,0.1,0.100027
DE,0.1,0.089756
DK,0.1,0.100021
ES,0.1,0.065456
FI,0.1,0.000496
FR,0.1,0.010613


In [7]:
# Use the constant from cbachela/qpmwp-course run to check the turnover
ASSIGNMENT_CONSTANT = 0.49954552248142037

print(
    f"Deviation vs Assignment Constant is satisfied: {np.isclose(turnover, ASSIGNMENT_CONSTANT, atol=1e-3)}"
)

Deviation vs Assignment Constant is satisfied: True
